# Imports

In [96]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error, r2_score
import pickle
import joblib

Mounting Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Preprocessing

In [3]:
male_players = pd.read_csv('/content/drive/My Drive/FIFA23_PREDICTIONMODEL/male_players (legacy).csv')

<ipython-input-3-9fa8cfb739a7>:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  male_players = pd.read_csv('/content/drive/My Drive/FIFA23_PREDICTIONMODEL/male_players (legacy).csv')


In [4]:
pd.set_option('display.max_columns', None)

In [5]:

male_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Columns: 110 entries, player_id to player_face_url
dtypes: float64(18), int64(45), object(47)
memory usage: 135.6+ MB


In [6]:
male_players.head()

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,league_id,league_name,league_level,club_team_id,club_name,club_position,club_jersey_number,club_loaned_from,club_joined_date,club_contract_valid_until_year,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,100500000.0,550000.0,27,1987-06-24,169,67,53.0,La Liga,1.0,241.0,FC Barcelona,CF,10.0,NaN,2004-07-01,2018.0,52,Argentina,1369.0,CF,10.0,Left,3,4,5,Medium/Low,Normal (170-),Yes,NaN,"#Speedster, #Dribbler, #FK Specialist, #Acroba...","Finesse Shot, Speed Dribbler (AI), One Club Pl...",93.0,89.0,86.0,96.0,27.0,63.0,84,94,71,89,85,96,89,90,76,96,96,90,94,94,95,80,73,77,60,88,48,22,92,90,76,NaN,25,21,20,6,11,15,14,8,NaN,89+3,89+3,89+3,92+3,90+3,90+3,90+3,92+3,92+3,92+3,92+3,90+3,79+3,79+3,79+3,90+3,62+3,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,79000000.0,375000.0,29,1985-02-05,185,80,53.0,La Liga,1.0,243.0,Real Madrid CF,LW,7.0,NaN,2009-07-01,2018.0,38,Portugal,1354.0,LW,7.0,Right,4,5,5,High/Low,Normal (185+),Yes,NaN,"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",93.0,93.0,81.0,91.0,32.0,79.0,83,95,86,82,87,93,88,79,72,92,91,94,93,90,63,94,94,89,79,93,63,24,91,81,85,NaN,22,31,23,7,11,15,14,11,NaN,91+1,91+1,91+1,89+3,91+1,91+1,91+1,89+3,89+3,89+3,89+3,87+3,77+3,77+3,77+3,87+3,63+3,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,54500000.0,275000.0,30,1984-01-23,180,80,19.0,Bundesliga,1.0,21.0,FC Bayern München,SUB,10.0,NaN,2009-08-28,2017.0,34,Netherlands,105035.0,RS,11.0,Left,2,4,5,High/Low,Normal (170-185),Yes,NaN,"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Diver, Injury Prone, Avoids Using Weaker Foot,...",93.0,86.0,83.0,92.0,32.0,64.0,80,85,50,86,86,93,85,83,76,90,93,93,93,89,91,86,61,78,65,90,47,39,89,84,80,NaN,29,26,26,10,8,11,5,15,NaN,84+3,84+3,84+3,88+2,87+3,87+3,87+3,88+2,88+2,88+2,88+2,87+3,78+3,78+3,78+3,87+3,64+3,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,52500000.0,275000.0,32,1981-10-03,195,95,16.0,Ligue 1,1.0,73.0,Paris Saint-Germain,ST,10.0,NaN,2012-07-01,2016.0,46,Sweden,1363.0,ST,10.0,Right,4,4,5,Medium/Low,Normal (185+),Yes,NaN,"#Poacher, #Aerial Threat, #Distance Shooter, #...","Power Free-Kick, Leadership, Flair, Long Shot ...",76.0,91.0,81.0,86.0,34.0,86.0,76,91,76,84

In [7]:
male_players.columns

Index(['player_id', 'player_url', 'fifa_version', 'fifa_update',
       'fifa_update_date', 'short_name', 'long_name', 'player_positions',
       'overall', 'potential',
       ...
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk',
       'player_face_url'],
      dtype='object', length=110)

In [8]:
male_players.describe()

,player_id,fifa_version,fifa_update,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_id,league_level,club_team_id,club_jersey_number,club_contract_valid_until_year,nationality_id,nation_team_id,nation_jersey_number,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,161583.000000,161583.000000,161583.0,161583.000000,161583.000000,1.595300e+05,159822.000000,161583.000000,161583.000000,161583.000000,159810.000000,158866.000000,159810.000000,159810.000000,159810.000000,161583.000000,9316.000000,9316.000000,161583.000000,161583.000000,161583.000000,1.036220e+05,143614.000000,143614.000000,143614.000000,143614.000000,143614.000000,143614.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.000000,161583.00000,161583.000000,128722.000000,161583.000000,161583.000000,161583.000000,161583.00000,161583.000000,161583.000000,161583.000000,161583.000000,17969.000000
mean,214484.722353,19.125514,2.0,65.699071,70.744008,2.326770e+06,10855.409768,25.123181,181.240205,75.235031,210.409017,1.380283,44914.264783,20.161323,2020.352418,55.570462,30414.309790,12.094354,2.939480,2.329905,1.107140,4.797334e+06,68.022672,52.230764,56.938175,62.081872,51.133323,64.814364,49.658875,45.617738,52.250552,58.362334,43.035121,55.220524,47.305230,43.059233,52.602254,58.206686,64.707420,64.879257,63.379149,61.576824,63.942531,56.719098,64.942816,63.090795,65.152374,47.152708,55.701757,46.583032,49.996460,53.06687,48.668492,57.816892,45.757957,47.669996,45.698588,16.52961,16.274918,16.140374,16.288861,16.636973,39.149090
std,34928.608856,2.559318,0.0,7.040855,6.259121,6.005746e+06,21941.656285,4.670207,6.750148,7.000456,444.685024,0.750646,53413.302607,16.870350,2.662184,47.891746,48296.823056,6.720156,0.662863,0.751792,0.383605,1.231500e+07,11.096042,13.921947,10.484536,10.301293,16.639606,9.737047,18.002046,19.227861,17.127721,14.685893,17.431618,18.593406,18.076438,17.228577,15.226187,16.566549,14.720619,14.472380,14.691549,9.170063,14.025247,15.522391,11.899687,15.820791,12.613079,19.010437,17.219537,20.381661,19.171899,14.26859,15.652208,12.329739,20.453699,21.336404,20.935273,17.67047,16.834294,16.476466,16.998697,17.980143,10.503788
min,2.000000,15.000000,2.0,40.000000,40.000000,1.000000e+03,500.000000,16.000000,154.000000,49.000000,1.000000,1.000000,1.000000,1.000000,2014.000000,1.000000,974.000000,1.000000,1.000000,1.000000,1.000000,9.000000e+03,21.000000,14.000000,20.000000,22.000000,14.000000,27.000000,5.000000,2.000000,4.000000,7.000000,3.000000,2.000000,4.000000,3.000000,5.000000,5.000000,11.000000,11.000000,11.000000,20.000000,10.000000,2.000000,13.000000,10.000000,12.000000,3.000000,2.000000,3.000000,2.000000,3.00000,5.000000,3.000000,1.000000,2.000000,3.000000,1.00000,1.000000,1.000000,1.000000,1.000000,12.000000
25%,199159.000000,17.000000,2.0,61.000000,66.000000,3.250000e+05,2000.000000,21.000000,176.000000,70.000000,19.000000,1.000000,450.000000,8.000000,2018.000000,21.000000,1343.000000,6.000000,3.000000,2.000000

In [9]:
print('The shape of this dataset is',male_players.shape[1])

The shape of this dataset is 110


Removing a percentage of null values in the dataset

In [10]:
threshold_percentage = 0.2  # 20% of the total rows
min_non_null_values = int(threshold_percentage * male_players.shape[0])
male_players.dropna(axis=1, thresh=min_non_null_values, inplace=True)

In [11]:
print('The shape of this dataset is',male_players.shape[1])

The shape of this dataset is 104


In [12]:
print(male_players.columns)

Index(['player_id', 'player_url', 'fifa_version', 'fifa_update',
       'fifa_update_date', 'short_name', 'long_name', 'player_positions',
       'overall', 'potential',
       ...
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk',
       'player_face_url'],
      dtype='object', length=104)


Listing useless variables

In [13]:
useless_columns = [
    "player_url",
    "player_id",
    "dob",
    "club_team_id",
    "long_name",
    "short_name",
    "real_face",
    "body_type",
    "club_jersey_number",
    "club_joined_date",
    "nationality_id",
    "player_face_url",
    "fifa_update",
    "fifa_update_date"
]


Dropping useless variables

In [14]:
new_players_22 = male_players.drop(columns=useless_columns)

print(new_players_22.columns)


Index(['fifa_version', 'player_positions', 'overall', 'potential', 'value_eur',
       'wage_eur', 'age', 'height_cm', 'weight_kg', 'league_id', 'league_name',
       'league_level', 'club_name', 'club_position',
       'club_contract_valid_until_year', 'nationality_name', 'preferred_foot',
       'weak_foot', 'skill_moves', 'international_reputation', 'work_rate',
       'release_clause_eur', 'player_traits', 'pace', 'shooting', 'passing',
       'dribbling', 'defending', 'physic', 'attacking_crossing',
       'attacking_finishing', 'attacking_heading_accuracy',
       'attacking_short_passing', 'attacking_volleys', 'skill_dribbling',
       'skill_curve', 'skill_fk_accuracy', 'skill_long_passing',
       'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed',
       'movement_agility', 'movement_reactions', 'movement_balance',
       'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength',
       'power_long_shots', 'mentality_aggression', 'mentality_in

In [15]:
new_players_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 90 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   fifa_version                    161583 non-null  int64  
 1   player_positions                161583 non-null  object 
 2   overall                         161583 non-null  int64  
 3   potential                       161583 non-null  int64  
 4   value_eur                       159530 non-null  float64
 5   wage_eur                        159822 non-null  float64
 6   age                             161583 non-null  int64  
 7   height_cm                       161583 non-null  int64  
 8   weight_kg                       161583 non-null  int64  
 9   league_id                       159810 non-null  float64
 10  league_name                     159810 non-null  object 
 11  league_level                    158866 non-null  float64
 12  club_name       

Splitting Numerical and Categorical variables in the dataset

In [16]:
categorical_data = pd.DataFrame()
numerical_data = pd.DataFrame()

In [17]:
for columns in new_players_22.columns:
    if new_players_22[columns].dtype == 'object':
        categorical_data[columns] = new_players_22[columns]
    else:
        numerical_data[columns] = new_players_22[columns]

In [18]:
categorical_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 35 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   player_positions  161583 non-null  object
 1   league_name       159810 non-null  object
 2   club_name         159810 non-null  object
 3   club_position     159810 non-null  object
 4   nationality_name  161583 non-null  object
 5   preferred_foot    161583 non-null  object
 6   work_rate         161583 non-null  object
 7   player_traits     73150 non-null   object
 8   ls                161583 non-null  object
 9   st                161583 non-null  object
 10  rs                161583 non-null  object
 11  lw                161583 non-null  object
 12  lf                161583 non-null  object
 13  cf                161583 non-null  object
 14  rf                161583 non-null  object
 15  rw                161583 non-null  object
 16  lam               161583 non-null  obj

In [19]:
numerical_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 55 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   fifa_version                    161583 non-null  int64  
 1   overall                         161583 non-null  int64  
 2   potential                       161583 non-null  int64  
 3   value_eur                       159530 non-null  float64
 4   wage_eur                        159822 non-null  float64
 5   age                             161583 non-null  int64  
 6   height_cm                       161583 non-null  int64  
 7   weight_kg                       161583 non-null  int64  
 8   league_id                       159810 non-null  float64
 9   league_level                    158866 non-null  float64
 10  club_contract_valid_until_year  159810 non-null  float64
 11  weak_foot                       161583 non-null  int64  
 12  skill_moves     

Checking for Null values in the numerical dataframe

In [20]:
print(numerical_data.isnull().sum())

fifa_version                          0
overall                               0
potential                             0
value_eur                          2053
wage_eur                           1761
age                                   0
height_cm                             0
weight_kg                             0
league_id                          1773
league_level                       2717
club_contract_valid_until_year     1773
weak_foot                             0
skill_moves                           0
international_reputation              0
release_clause_eur                57961
pace                              17969
shooting                          17969
passing                           17969
dribbling                         17969
defending                         17969
physic                            17969
attacking_crossing                    0
attacking_finishing                   0
attacking_heading_accuracy            0
attacking_short_passing               0


Replacing null values with the mean of each column (mean ffilling)



In [21]:
numerical_data.ffill(axis=0, inplace=True)

In [22]:
print(numerical_data.isnull().sum())

fifa_version                          0
overall                               0
potential                             0
value_eur                             0
wage_eur                              0
age                                   0
height_cm                             0
weight_kg                             0
league_id                             0
league_level                          0
club_contract_valid_until_year        0
weak_foot                             0
skill_moves                           0
international_reputation              0
release_clause_eur                50427
pace                                  0
shooting                              0
passing                               0
dribbling                             0
defending                             0
physic                                0
attacking_crossing                    0
attacking_finishing                   0
attacking_heading_accuracy            0
attacking_short_passing               0


In [23]:
numerical_data['release_clause_eur'].fillna(numerical_data['release_clause_eur'].mean(), inplace=True)
numerical_data['mentality_composure'].fillna(numerical_data['mentality_composure'].mean(), inplace=True)


In [24]:
print(numerical_data.isnull().sum())

fifa_version                      0
overall                           0
potential                         0
value_eur                         0
wage_eur                          0
age                               0
height_cm                         0
weight_kg                         0
league_id                         0
league_level                      0
club_contract_valid_until_year    0
weak_foot                         0
skill_moves                       0
international_reputation          0
release_clause_eur                0
pace                              0
shooting                          0
passing                           0
dribbling                         0
defending                         0
physic                            0
attacking_crossing                0
attacking_finishing               0
attacking_heading_accuracy        0
attacking_short_passing           0
attacking_volleys                 0
skill_dribbling                   0
skill_curve                 

Ffilling categorical values with the mode

In [25]:
categorical_data.ffill(axis=0, inplace=True)

In [26]:
print(categorical_data.isnull().sum())

player_positions    0
league_name         0
club_name           0
club_position       0
nationality_name    0
preferred_foot      0
work_rate           0
player_traits       0
ls                  0
st                  0
rs                  0
lw                  0
lf                  0
cf                  0
rf                  0
rw                  0
lam                 0
cam                 0
ram                 0
lm                  0
lcm                 0
cm                  0
rcm                 0
rm                  0
lwb                 0
ldm                 0
cdm                 0
rdm                 0
rwb                 0
lb                  0
lcb                 0
cb                  0
rcb                 0
rb                  0
gk                  0
dtype: int64


Encoding categorical values using LabelEncoder due to its simplicity

In [27]:
encoder = LabelEncoder()

In [28]:
encoded_categroical_data = categorical_data.copy()

In [29]:
categorical_data = categorical_data.astype(str)

In [30]:
for columns in categorical_data.columns:
    encoded_categroical_data[columns] = encoder.fit_transform(categorical_data[columns])

# encoded_categorical_data = categorical_data.apply(encoder.fit_transform)

In [31]:
encoded_categroical_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 35 columns):
 #   Column            Non-Null Count   Dtype
---  ------            --------------   -----
 0   player_positions  161583 non-null  int64
 1   league_name       161583 non-null  int64
 2   club_name         161583 non-null  int64
 3   club_position     161583 non-null  int64
 4   nationality_name  161583 non-null  int64
 5   preferred_foot    161583 non-null  int64
 6   work_rate         161583 non-null  int64
 7   player_traits     161583 non-null  int64
 8   ls                161583 non-null  int64
 9   st                161583 non-null  int64
 10  rs                161583 non-null  int64
 11  lw                161583 non-null  int64
 12  lf                161583 non-null  int64
 13  cf                161583 non-null  int64
 14  rf                161583 non-null  int64
 15  rw                161583 non-null  int64
 16  lam               161583 non-null  int64
 17  cam       

In [32]:
encoded_categroical_data

,player_positions,league_name,club_name,club_position,nationality_name,preferred_foot,work_rate,player_traits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,412,15,478,3,6,0,7,1128,392,392,392,335,300,300,300,335,432,432,432,413,330,330,330,413,204,216,216,216,204,151,111,111,111,151,25
1,1128,15,1013,14,134,1,1,3066,396,396,396,327,302,302,302,327,425,425,425,402,315,315,315,402,215,228,228,228,215,170,155,155,155,170,29
2,1541,6,481,28,117,0,1,550,370,370,370,323,291,291,291,323,421,421,421,402,321,321,321,402,225,238,238,238,225,156,116,116,116,156,20
3,1838,26,947,27,161,1,7,3163,385,385,385,307,286,286,286,307,410,410,410,384,306,306,306,384,195,249,249,249,195,163,178,178,178,163,34
4,719,6,481,5,64,1,8,1341,84,84,84,50,48,48,48,50,73,73,73,81,72,72,72,81,77,87,87,87,77,79,82,82,82,79,252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,1643,40,663,21,34,1,8,1609,115,115,115,68,62,62,62,68,105,105,105,114,79,79,79,114,81,54,54,54,81,78,52,52,52,78,19
161579,721,40,633,21,34,0,8,1609,69,69,69,41,35,35,35,41,56,56,56,68,63,63,63,68,101,90,90,90,101,106,127,127,127,106,24
161580,289,31,1165,21,137,1,2,1609,111,111,111,68,62,62,62,68,109,109,109,118,103,103,103,118,115,108,108,108,115,118,115,115,115,118,19
161581,1694,31,1139,21,137,1,8,1609,115,115,115,68,62,62,62,68,109,109,109,105,83,83,83,105,68,46,46,46,68,61,40,40,40,61,24


In [33]:
clean_male_players_22_data = pd.concat([numerical_data, encoded_categroical_data], axis=1)

In [34]:
clean_male_players_22_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 90 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   fifa_version                    161583 non-null  int64  
 1   overall                         161583 non-null  int64  
 2   potential                       161583 non-null  int64  
 3   value_eur                       161583 non-null  float64
 4   wage_eur                        161583 non-null  float64
 5   age                             161583 non-null  int64  
 6   height_cm                       161583 non-null  int64  
 7   weight_kg                       161583 non-null  int64  
 8   league_id                       161583 non-null  float64
 9   league_level                    161583 non-null  float64
 10  club_contract_valid_until_year  161583 non-null  float64
 11  weak_foot                       161583 non-null  int64  
 12  skill_moves     

# Feature engineering

In [35]:
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

Finding the correlation of the Quantitative variables

In [36]:
# qdf = new_players_22.select_dtypes(include=['int64','float64'])

In [37]:
# qdf.head()

In [38]:
clean_male_players_22_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 90 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   fifa_version                    161583 non-null  int64  
 1   overall                         161583 non-null  int64  
 2   potential                       161583 non-null  int64  
 3   value_eur                       161583 non-null  float64
 4   wage_eur                        161583 non-null  float64
 5   age                             161583 non-null  int64  
 6   height_cm                       161583 non-null  int64  
 7   weight_kg                       161583 non-null  int64  
 8   league_id                       161583 non-null  float64
 9   league_level                    161583 non-null  float64
 10  club_contract_valid_until_year  161583 non-null  float64
 11  weak_foot                       161583 non-null  int64  
 12  skill_moves     

In [39]:
clean_male_players_22_data['gk'] = pd.to_numeric(clean_male_players_22_data['gk'], errors='coerce')

In [40]:
corr_matrix = clean_male_players_22_data.corr()

In [41]:
corr_matrix

,fifa_version,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_id,league_level,club_contract_valid_until_year,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,player_positions,league_name,club_name,club_position,nationality_name,preferred_foot,work_rate,player_traits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
fifa_version,1.000000,0.047487,0.092656,0.088918,-0.086446,0.031996,0.018028,-0.019338,0.182658,0.000612,0.825389,0.008250,0.043317,-0.030201,0.014927,-0.002077,0.015393,0.041228,0.054551,0.034171,0.001539,-0.006268,0.011061,-0.023493,0.037100,-0.028777,0.023575,0.000619,-0.031278,0.030289,0.015432,-0.016729,-0.024547,0.005541,0.006954,0.002432,0.053602,-0.012627,-0.011769,-0.007094,-0.024680,-0.005321,-0.003430,0.012214,0.065872,-0.040897,0.023675,0.042991,0.010594,0.005122,-0.000154,0.002465,0.002967,0.002688,-0.000894,-0.001841,-0.009908,0.017273,0.000986,0.002216,-0.009719,-0.060526,0.085264,0.034029,0.034029,0.034029,0.019150,0.033420,0.033420,0.033420,0.019150,0.039143,0.039143,0.039143,0.028467,0.046992,0.046992,0.046992,0.028467,0.033396,0.030416,0.030416,0.030416,0.033396,0.029602,0.022171,0.022171,0.022171,0.029602,0.036577
overall,0.047487,1.000000,0.695362,0.566842,0.611123,0.458390,0.044415,0.140015,-0.152389,-0.222534,0.072574,0.217422,0.330951,0.485234,0.475343,0.184880,0.487534,0.685266,0.610898,0.331512,0.520416,0.396833,0.326376,0.340029,0.499757,0.377048,0.369827,0.415863,0.387269,0.483808,0.457011,0.194625,0.208990,0.263671,0.845753,0.099129,0.480297,0.275578,0.360025,0.350786,0.409086,0.398310,0.314282,0.350178,0.493282,0.334488,0.604074,0.274173,0.250624,0.220398,-0.013603,-0.010862,-0.015521,-0.004731,-0.011783,-0.015008,0.051884,0.011962,-0.213040,-0.035847,-0.042408,-0.223711,-0.051113,0.552396,0.552396,0.552396,0.572444,0.587947,0.587947,0.587947,0.572444,0.567905,0.567905,0.567905,0.558515,0.607929,0.607929,0.607929,0.558515,0.532344,0.544101,0.544101,0.544101,0.532344,0.498317,0.438253,0.438253,0.438253,0.498317,0.086577
potential,0.092656,0.695362,1.000000,0.533645,0.511002,-0.201506,0.006744,-0.008111,-0.195047,-0.194708,0.157952,0.167492,0.285126,0.367256,0.447948,0.259561,0.322981,0.461646,0.489407,0.168780,0.202931,0.264256,0.245701,0.208889,0.383164,0.256491,0.324038,0.292871,0.235827,0.341345,0.366798,0.240387,0.244990,0.235361,0.540548,0.131958,0.326665,0.129966,0.212416,0.099452,0.270333,0.197977,0.163596,0.253592,0.358591,0.225312,0.370952,0.152613,0.152078,0.134847,-0.044826,-0.044119,-0.048803,-0.042978,-0.044156,-0.015351,0.011382,0.016104,-0.105728,-0.019365,-0.035255,-0.151125,-0.016999,0.405186,0.405186,0.405186,0.447870,0.454070,0.454070,0.454070,0.447870,0.439701,0.439701,0.439701,0.431729,0.444070,0.444070,0.444070,0.431729,0.371794,0.359022,0.359022,0.359022,0.371794,0.339317,0.263782,0.263782,0.263782,0.339317,0.018625
value_eur,0.088918,0.566842,0.533645,1.000000,0.775954,0.069369,0.010930,0.036085,-0.087801,-0.132948,0.152107,0.147786,0.251753,0.599440,0.906883,0.171658,0.302730,0.396404,0.385383,0.133605,0.212193,0.224838,0.223635,0.164210,0.294343,0.245879,0.241548,0.254912,0.222627,0.275380,0.272726,0.154425,0.158787,0.174569,0.490082,0.102973,0.270648,0.119820,0.189634,0.125

In [42]:
corr_matrix['overall']

fifa_version    0.047487
overall         1.000000
potential       0.695362
value_eur       0.566842
wage_eur        0.611123
                  ...   
lcb             0.438253
cb              0.438253
rcb             0.438253
rb              0.498317
gk              0.086577
Name: overall, Length: 90, dtype: float64

Finding features that have a strong correlation with overall

In [43]:
corr_with_overall = corr_matrix['overall'].sort_values(ascending=False)
print("Features strongly correlated with overall rating:")
print(corr_with_overall)

Features strongly correlated with overall rating:
overall               1.000000
movement_reactions    0.845753
potential             0.695362
passing               0.685266
wage_eur              0.611123
                        ...   
player_traits        -0.051113
league_id            -0.152389
club_position        -0.213040
league_level         -0.222534
work_rate            -0.223711
Name: overall, Length: 90, dtype: float64


Using the random forest classsifier to find the feature importance

In [44]:
#Choosing X and Y
X= clean_male_players_22_data.drop('overall',axis=1)
y=clean_male_players_22_data['overall']

In [45]:
X

,fifa_version,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_id,league_level,club_contract_valid_until_year,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,player_positions,league_name,club_name,club_position,nationality_name,preferred_foot,work_rate,player_traits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,15,95,100500000.0,550000.0,27,169,67,53.0,1.0,2018.0,3,4,5,4.788777e+06,93.0,89.0,86.0,96.0,27.0,63.0,84,94,71,89,85,96,89,90,76,96,96,90,94,94,95,80,73,77,60,88,48,22,92,90,76,57.816892,25,21,20,6,11,15,14,8,412,15,478,3,6,0,7,1128,392,392,392,335,300,300,300,335,432,432,432,413,330,330,330,413,204,216,216,216,204,151,111,111,111,151,25
1,15,92,79000000.0,375000.0,29,185,80,53.0,1.0,2018.0,4,5,5,4.788777e+06,93.0,93.0,81.0,91.0,32.0,79.0,83,95,86,82,87,93,88,79,72,92,91,94,93,90,63,94,94,89,79,93,63,24,91,81,85,57.816892,22,31,23,7,11,15,14,11,1128,15,1013,14,134,1,1,3066,396,396,396,327,302,302,302,327,425,425,425,402,315,315,315,402,215,228,228,228,215,170,155,155,155,170,29
2,15,90,54500000.0,275000.0,30,180,80,19.0,1.0,2017.0,2,4,5,4.788777e+06,93.0,86.0,83.0,92.0,32.0,64.0,80,85,50,86,86,93,85,83,76,90,93,93,93,89,91,86,61,78,65,90,47,39,89,84,80,57.816892,29,26,26,10,8,11,5,15,1541,6,481,28,117,0,1,550,370,370,370,323,291,291,291,323,421,421,421,402,321,321,321,402,225,238,238,238,225,156,116,116,116,156,20
3,15,90,52500000.0,275000.0,32,195,95,16.0,1.0,2016.0,4,4,5,4.788777e+06,76.0,91.0,81.0,86.0,34.0,86.0,76,91,76,84,92,88,80,80,76,90,74,77,86,85,41,93,72,78,93,88,84,20,86,83,91,57.816892,25,41,27,13,15,10,9,12,1838,26,947,27,161,1,7,3163,385,385,385,307,286,286,286,307,410,410,410,384,306,306,306,384,195,249,249,249,195,163,178,178,178,163,34
4,15,90,63500000.0,300000.0,28,193,92,19.0,1.0,2019.0,4,1,5,4.788777e+06,76.0,91.0,81.0,86.0,34.0,86.0,25,25,25,42,25,25,25,25,41,31,58,61,43,89,35,42,78,44,83,25,29,30,25,20,37,57.816892,25,25,25,87,85,92,90,86,719,6,481,5,64,1,8,1341,84,84,84,50,48,48,48,50,73,73,73,81,72,72,72,81,77,87,87,87,77,79,82,82,82,79,252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,23,61,110000.0,700.0,18,180,73,2012.0,1.0,2024.0,3,2,1,2.390000e+05,63.0,48.0,40.0,47.0,24.0,44.0,48,49,36,38,43,47,39,35,33,45,67,60,54,39,66,64,41,42,52,31,26,21,40,42,54,40.000000,23,21,25,9,13,13,12,7,1643,40,663,21,34,1,8,1609,115,115,115,68,62,62,62,68,105,105,105,114,79,79,79,114,81,54,54,54,81,78,52,52,52,78,19
161579,23,58,110000.0,750.0,19,188,83,2012.0,1.0,2027.0,3,2,1,1.930000e+05,55.0,25.0,29.0,34.0,48.0,57.0,30,19,42,30,25,30,24,26,25,32,54,56,45,42,48,33,56,53,64,25,46,46,31,31,31,35.000000,50,51,45,6,14,8,13,14,721,40,633,21,34,0,8,1609,69,69,69,41,35,35,35,41,56,56,56,68,63,63,63,68,101,90,90,90,101,106,127,127,127,106,24
161580,23,58,110000.0,500.0,19,181,73,65.0,1.0,2023.0,2,2,1,1.930000e+05,65.0,36.0,43.0,46.0,43.0,53.0,37,30,47,51,37,49,30,30,43,35,66,64,58,50,65,51,52,53,53,32,51,43,53,40,37,

In [46]:
y

0         93
1         92
2         90
3         90
4         90
          ..
161578    46
161579    46
161580    46
161581    46
161582    46
Name: overall, Length: 161583, dtype: int64

In [47]:
#Using test_train split to split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [49]:
#Determining the feature importance of dependent variables using Random forest classifier

In [50]:
feature_impotance = pd.DataFrame({'feature': X.columns, 'importance': rf.feature_importances_})

In [51]:
feature_impotance

,feature,importance
0,fifa_version,0.019142
1,potential,0.045405
2,value_eur,0.045674
3,wage_eur,0.011846
4,age,0.017233
...,...,...
84,lcb,0.032600
85,cb,0.033103
86,rcb,0.031273
87,rb,0.023844


In [52]:
feature_impotance = feature_impotance.sort_values('importance', ascending=False)
feature_impotance

,feature,importance
2,value_eur,0.045674
1,potential,0.045405
85,cb,0.033103
84,lcb,0.032600
88,gk,0.032470
...,...,...
10,weak_foot,0.001728
8,league_level,0.001455
11,skill_moves,0.001087
12,international_reputation,0.000941


In [53]:
top_10_features = corr_with_overall.iloc[1:21]

print("Top 10 features strongly correlated with overall rating:")
print(top_10_features)

Top 10 features strongly correlated with overall rating:
movement_reactions     0.845753
potential              0.695362
passing                0.685266
wage_eur               0.611123
dribbling              0.610898
lcm                    0.607929
rcm                    0.607929
cm                     0.607929
mentality_composure    0.604074
rf                     0.587947
cf                     0.587947
lf                     0.587947
lw                     0.572444
rw                     0.572444
lam                    0.567905
ram                    0.567905
cam                    0.567905
value_eur              0.566842
rm                     0.558515
lm                     0.558515
Name: overall, dtype: float64


In [54]:
#Picking the important features according to the feature importance and correlation matrix
selected_features=["movement_reactions","potential","passing","wage_eur","dribbling","value_eur","player_positions"]

# Training, Testing and Evaluation



In [55]:
X = clean_male_players_22_data[selected_features]
y = pd.DataFrame(clean_male_players_22_data['overall'])

In [56]:
X

,movement_reactions,potential,passing,wage_eur,dribbling,value_eur,player_positions
0,94,95,86.0,550000.0,96.0,100500000.0,412
1,90,92,81.0,375000.0,91.0,79000000.0,1128
2,89,90,83.0,275000.0,92.0,54500000.0,1541
3,85,90,81.0,275000.0,86.0,52500000.0,1838
4,89,90,81.0,300000.0,86.0,63500000.0,719
...,...,...,...,...,...,...,...
161578,39,61,40.0,700.0,47.0,110000.0,1643
161579,42,58,29.0,750.0,34.0,110000.0,721
161580,50,58,43.0,500.0,46.0,110000.0,289
161581,45,70,36.0,500.0,46.0,150000.0,1694


In [57]:
y

,overall
0,93
1,92
2,90
3,90
4,90
...,...
161578,46
161579,46
161580,46
161581,46


In [58]:
#Scaling using the standard scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

## Training - RandomForestRegressor

In [60]:
rf_regressor = RandomForestRegressor(n_estimators=112, random_state=42)

In [61]:
rf_regressor.fit(X_train, y_train)

<ipython-input-61-56a44efaf177>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_regressor.fit(X_train, y_train)


RandomForestRegressor(n_estimators=112, random_state=42)

In [62]:
initial_prediction = rf_regressor.predict(X_test)
initial_prediction


array([66.75892857, 65.47767857, 69.39285714, ..., 56.6875    ,
       63.97321429, 53.84821429])

In [63]:
initial_rmse = np.sqrt(mean_squared_error(y_test, initial_prediction))
print("Initial RMSE:", initial_rmse)

Initial RMSE: 1.4173254199013734


In [68]:
param_grid = {
    'n_estimators': [100, 120],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

In [69]:
scoring = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)
scoring

make_scorer(<lambda>, greater_is_better=False)

In [70]:
grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid,
                           scoring=scoring, cv=5, verbose=1, n_jobs=-1)

In [71]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=5,
             estimator=RandomForestRegressor(n_estimators=112, random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [None, 10], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 120]},
             scoring=make_scorer(<lambda>, greater_is_better=False), verbose=1)

In [72]:
best_parameter = grid_search.best_params_
best_parameter

{'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 120}

In [73]:
best_model = grid_search.best_estimator_
best_model

RandomForestRegressor(n_estimators=120, random_state=42)

In [102]:
#training the RandomForestRegressor using the best parameters from
imporoved_rfregressor = RandomForestRegressor(**best_parameter)

In [103]:
imporoved_rfregressor.fit(X_train, y_train)

<ipython-input-103-eba9eb5dd05b>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  imporoved_rfregressor.fit(X_train, y_train)


RandomForestRegressor(n_estimators=120)

In [74]:
scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring=scoring)
scores

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed whe

array([-1.42982246, -1.41961421, -1.42353838, -1.43552552, -1.42439698])

In [75]:
rmse_score =np.sqrt(-scores)
rmse_score

array([1.19575184, 1.19147564, 1.19312128, 1.19813418, 1.19348103])

In [104]:
improved_model_prediction = imporoved_rfregressor.predict(X_test)
pd.DataFrame(improved_model_prediction)

,0
0,64.600000
1,65.600000
2,69.516667
3,63.491667
4,68.741667
...,...
32312,78.158333
32313,64.025000
32314,56.850000
32315,64.100000


In [89]:
y_test.iloc[100:]

,overall
141448,56
94625,67
152654,65
113491,66
53609,73
...,...
143827,78
135022,64
84969,56
26538,63


In [106]:
rsme_improved = np.sqrt(mean_squared_error(y_test, improved_model_prediction))
print("Improved RMSE:", rsme_improved)

Improved RMSE: 1.415433575818197


In [99]:
mae = mean_absolute_error(y_test, improved_model_prediction)
print("mean squared error:", mae)

mean squared error: 0.9439282460310093


In [100]:
r2 = r2_score(y_test, improved_model_prediction)
print("r_squared", r2)

r_squared 0.9595830422036872


In [107]:
imporoved_rfregressor.score(X_test, y_test)

0.9595781026460589

## Training - XGBoost Model

In [108]:
#using the xgboost regressor
xgboost_initial_model = xgb.XGBRegressor()

In [109]:
xgboost_initial_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [110]:
xgboost_initial_prediction = xgboost_initial_model.predict(X_test)
xgboost_initial_prediction

array([68.075096, 64.80852 , 69.629906, ..., 56.867638, 63.92332 ,
       53.019474], dtype=float32)

In [111]:
xgboost_initial_rmse = np.sqrt(mean_squared_error(y_test, xgboost_initial_prediction))
print("Initial RMSE:", xgboost_initial_rmse)

Initial RMSE: 1.44085460162583


In [113]:
#Hyperparameter tuning
xgb_params = {
    'max_depth': [3, 4],
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100, 150],
    'colsample_bytree': [0.8, 0.9],
    'subsample': [0.8, 0.9]
}

In [124]:
xgb_grid_search = GridSearchCV(estimator=xgboost_initial_model, param_grid=xgb_params, scoring=scoring, cv=5, verbose=1, n_jobs=-1)

In [125]:
xgb_grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 0.9],
                         'learning_rate': [0.1, 0.01], 'max_depth': [3, 4],
                         'n_estimators': [100, 150], 'subsample': [0.8, 0.9]},
             scoring=make_scorer(<lambda>, greater_is_better=False), verbose=1)

In [126]:
xgb_best_params = xgb_grid_search.best_params_
xgb_best_params

{'colsample_bytree': 0.8,
 'learning_rate': 0.1,
 'max_depth': 4,
 'n_estimators': 150,
 'subsample': 0.8}

In [127]:
xgb_best_model = xgb_grid_search.best_estimator_
xgb_best_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=150, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [128]:
xgb_final_model = xgb.XGBRegressor(**xgb_best_params)

In [129]:
xgb_final_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=150, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [130]:
xgb_final_prediction = xgb_final_model.predict(X_test)
xgb_final_prediction

array([67.22086, 64.44372, 69.08074, ..., 57.81616, 64.40123, 52.66626],
      dtype=float32)

In [131]:
xgb_final_rmse = np.sqrt(mean_squared_error(y_test, xgb_final_prediction))
print("Final RMSE:", xgb_final_rmse)

Final RMSE: 1.5405102678692997


In [132]:
xgb_mae = mean_absolute_error(y_test, xgb_final_prediction)
print("mean squared error:", xgb_mae)

mean squared error: 1.1506523899347854


In [133]:
xgb_r2 = r2_score(y_test, xgb_final_prediction)
print("r_squared", xgb_r2)

r_squared 0.9521185930996207


In [134]:
xgb_final_model.score(X_test, y_test)

0.9521185930996207

## Training - Gradient Boost Regression

In [135]:
gradient_boost_initial_model = GradientBoostingRegressor()

In [136]:
gradient_boost_initial_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor()

In [137]:
gradient_boost_predict = gradient_boost_initial_model.predict(X_test)
gradient_boost_predict

array([66.81169891, 63.9959379 , 69.15013324, ..., 56.98699883,
       64.87127968, 52.88977548])

In [138]:
gradient_boost_initial_rmse = np.sqrt(mean_squared_error(y_test, gradient_boost_predict))
print("Initial RMSE:", gradient_boost_initial_rmse)

Initial RMSE: 1.679130309605731


In [139]:
gradient_boost_params = {
    'n_estimators': [100, 150],
    'learning_rate': [0.1, 0.01],
    'max_depth': [3, 4],
    'min_samples_split': [2, 3],
    'min_samples_leaf': [1, 2]
}

In [140]:
gradient_boost_grid_search = GridSearchCV(estimator=gradient_boost_initial_model, param_grid=gradient_boost_params, scoring=scoring, cv=5, verbose=1, n_jobs=-1)

In [141]:
              gradient_boost_grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.01], 'max_depth': [3, 4],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 3],
                         'n_estimators': [100, 150]},
             scoring=make_scorer(<lambda>, greater_is_better=False), verbose=1)

In [142]:
gradient_boost_best_params = gradient_boost_grid_search.best_params_
gradient_boost_best_params

{'learning_rate': 0.1,
 'max_depth': 4,
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 150}

In [143]:
gradient_boost_best_model = gradient_boost_grid_search.best_estimator_
gradient_boost_best_model

GradientBoostingRegressor(max_depth=4, min_samples_split=3, n_estimators=150)

In [144]:
gradient_boost_final_model = GradientBoostingRegressor(**gradient_boost_best_params)

In [145]:
gradient_boost_final_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(max_depth=4, min_samples_split=3, n_estimators=150)

In [146]:
gradient_boost_final_prediction = gradient_boost_final_model.predict(X_test)
gradient_boost_final_prediction

array([67.88773534, 64.4922728 , 69.12462167, ..., 57.86794807,
       64.12836018, 52.63756217])

In [147]:
gradient_boost_final_rmse = np.sqrt(mean_squared_error(y_test, gradient_boost_final_prediction))
print("Final RMSE:", gradient_boost_final_rmse)

Final RMSE: 1.5461325323365387


In [148]:
gradient_boost_mae = mean_absolute_error(y_test, gradient_boost_final_prediction)
print("mean squared error:", gradient_boost_mae)

mean squared error: 1.1539142070627149


In [149]:
gradient_boost_r2 = r2_score(y_test, gradient_boost_final_prediction)
print("r_squared", gradient_boost_r2)

r_squared 0.9517684582390373


In [150]:
gradient_boost_score = gradient_boost_final_model.score(X_test, y_test)
gradient_boost_score

0.9517684582390373

# Model Scores



1.   Random Forest Regressor: 0.9595781026460589
2.   Xgboost Regressor:       0.9521185930996207
3.   Gradient Boost Regresor: 0.9517684582390373


# Training model with unseen data:players_22-1 dataset

***Cleaning new data***

In [163]:
players_22 = pd.read_csv('/content/drive/My Drive/FIFA23_PREDICTIONMODEL/players_22-1.csv')

<ipython-input-163-b31c0e1ad034>:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players_22 = pd.read_csv('/content/drive/My Drive/FIFA23_PREDICTIONMODEL/players_22-1.csv')


In [164]:
pd.set_option('display.max_columns', None)

In [165]:
players_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 16.1+ MB


In [166]:
players_22.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,1987-06-24,170,72,73.0,Paris Saint-Germain,French Ligue 1,1.0,RW,30.0,NaN,2021-08-10,2023.0,52,Argentina,1369.0,RW,10.0,Left,4,4,5,Medium/Low,Unique,Yes,144300000.0,"#Dribbler, #Distance Shooter, #FK Specialist, ...","Finesse Shot, Long Shot Taker (AI), Playmaker ...",85.0,92.0,91.0,95.0,34.0,65.0,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96,20,35,24,6,11,15,14,8,NaN,89+3,89+3,89+3,92,93,93,93,92,93,93,93,91+2,87+3,87+3,87+3,91+2,66+3,64+3,64+3,64+3,66+3,61+3,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,1988-08-21,185,81,21.0,FC Bayern München,German 1. Bundesliga,1.0,ST,9.0,NaN,2014-07-01,2023.0,37,Poland,1353.0,RS,9.0,Right,4,4,5,High/Medium,Unique,Yes,197200000.0,"#Aerial Threat, #Distance Shooter, #Clinical F...","Solid Player, Finesse Shot, Outside Foot Shot,...",78.0,92.0,79.0,86.0,44.0,82.0,71,95,90,85,89,85,79,85,70,88,77,79,77,93,82,90,85,76,86,87,81,49,95,81,90,88,35,42,19,15,6,12,8,10,NaN,90+2,90+2,90+2,85,88,88,88,85,86+3,86+3,86+3,84+3,80+3,80+3,80+3,84+3,64+3,66+3,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,1985-02-05,187,83,11.0,Manchester United,English Premier League,1.0,ST,7.0,NaN,2021-08-27,2023.0,38,Portugal,1354.0,ST,7.0,Right,4,5,5,High/Low,Unique,Yes,83300000.0,"#Aerial Threat, #Dribbler, #Distance Shooter, ...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",87.0,94.0,80.0,88.0,34.0,75.0,87,95,90,80,86,88,81,84,77,88,85,88,86,94,74,94,95,77,77,93,63,29,95,76,88,95,24,32,24,7,11,15,14,11,NaN,90+1,90+1,90+1,88,89,89,89,88,86+3,86+3,86+3,86+3,78+3,78+3,78+3,86+3,63+3,59+3,59+3,59+3,63+3,60+3,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png


In [167]:
players_22.columns

Index(['sofifa_id', 'player_url', 'short_name', 'long_name',
       'player_positions', 'overall', 'potential', 'value_eur', 'wage_eur',
       'age',
       ...
       'lcb', 'cb', 'rcb', 'rb', 'gk', 'player_face_url', 'club_logo_url',
       'club_flag_url', 'nation_logo_url', 'nation_flag_url'],
      dtype='object', length=110)

In [168]:
players_22.describe()

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_jersey_number,club_contract_valid_until,nationality_id,nation_team_id,nation_jersey_number,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,19239.000000,19239.000000,19239.000000,1.916500e+04,19178.000000,19239.000000,19239.000000,19239.000000,19178.000000,19178.000000,19178.000000,19178.000000,19239.000000,759.000000,759.000000,19239.000000,19239.000000,19239.000000,1.806300e+04,17107.000000,17107.000000,17107.000000,17107.000000,17107.000000,17107.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,2132.000000
mean,231468.086959,65.772182,71.079370,2.850452e+06,9017.989363,25.210822,181.299704,74.943032,50580.498123,1.354364,20.945250,2022.764000,58.602682,14480.848485,12.567852,2.946151,2.352461,1.094184,5.374044e+06,68.213071,52.345297,57.312562,62.561174,51.703630,64.823289,49.577421,45.894433,51.783877,58.867977,42.463849,55.660429,47.268933,42.249025,53.072249,58.472010,64.652893,64.714902,63.500078,61.450023,64.068611,57.776860,64.813504,63.084880,65.007745,46.642705,55.538957,46.613545,50.330215,53.964603,47.858724,57.929830,46.601746,48.045584,45.906700,16.406102,16.192474,16.055356,16.229274,16.491814,36.439962
std,27039.717497,6.880232,6.086213,7.613700e+06,19470.176724,4.748235,6.863179,7.069434,54401.868535,0.747865,17.909369,1.213203,50.298614,35328.730217,7.039116,0.671560,0.767659,0.371098,1.494837e+07,10.933155,14.051623,10.068965,9.651312,16.189746,9.791886,18.034661,19.721023,17.294183,14.490858,17.653329,18.784590,18.181085,17.178590,15.026569,16.663722,15.167399,14.965426,14.862285,9.042281,14.324789,13.192224,12.122977,16.145279,12.663518,19.411583,16.972181,20.677077,19.621601,13.650481,15.768583,12.159326,20.200807,21.232718,20.755683,17.574028,16.839528,16.564554,17.059779,17.884833,10.751563
min,41.000000,47.000000,49.000000,9.000000e+03,500.000000,16.000000,155.000000,49.000000,1.000000,1.000000,1.000000,2021.000000,1.000000,1318.000000,1.000000,1.000000,1.000000,1.000000,1.600000e+04,28.000000,18.000000,25.000000,27.000000,14.000000,29.000000,6.000000,2.000000,5.000000,7.000000,3.000000,4.000000,6.000000,4.000000,9.000000,8.000000,14.000000,15.000000,18.000000,25.000000,15.000000,20.000000,22.000000,12.000000,19.000000,4.000000,10.000000,3.000000,2.000000,10.000000,7.000000,12.000000,4.000000,5.000000,5.000000,2.000000,2.000000,2.000000,2.000000,2.000000,15.000000
25%,214413.500000,61.000000,67.000000,4.750000e+05,1000.000000,21.000000,176.000000,70.000000,479.000000,1.000000,9.000000,2022.000000,21.000000,1338.000000,7.000000,3.000000,2.000000,1.000000,8.060000e+05,62.000000,42.000000,51.000000,57.000000,37.000000,59.000000,38.000000,30.000000,44.000000,54.000000,30.000000,50.000000,35.000000,31.000000,44.000000,55.000000,57.000000,58.000000,55.000000,56.000

In [169]:
print('The shape of this dataset is',players_22.shape[1])

The shape of this dataset is 110


In [170]:
threshold_percentage = 0.2  # 20% of the total rows
min_non_null_values = int(threshold_percentage * players_22.shape[0])
players_22.dropna(axis=1, thresh=min_non_null_values, inplace=True)

In [171]:
print('The shape of this dataset is',players_22.shape[1])

The shape of this dataset is 103


In [174]:
unnecessary_columns = [
    "player_url",
    "sofifa_id",
    "dob",
    "club_team_id",
    "long_name",
    "real_face",
    "body_type",
    "club_jersey_number",
    "club_joined",
    "nationality_id",
    "player_face_url",
    "club_logo_url",
    "club_flag_url",
    "nation_flag_url"
]

print(unnecessary_columns)

['player_url', 'sofifa_id', 'dob', 'club_team_id', 'long_name', 'real_face', 'body_type', 'club_jersey_number', 'club_joined', 'nationality_id', 'player_face_url', 'club_logo_url', 'club_flag_url', 'nation_flag_url']


In [175]:
new_22 = players_22.drop(columns=unnecessary_columns)

print(new_22.columns)

Index(['short_name', 'player_positions', 'overall', 'potential', 'value_eur',
       'wage_eur', 'age', 'height_cm', 'weight_kg', 'club_name', 'league_name',
       'league_level', 'club_position', 'club_contract_valid_until',
       'nationality_name', 'preferred_foot', 'weak_foot', 'skill_moves',
       'international_reputation', 'work_rate', 'release_clause_eur',
       'player_traits', 'pace', 'shooting', 'passing', 'dribbling',
       'defending', 'physic', 'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
      

In [176]:
new_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 89 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   short_name                   19239 non-null  object 
 1   player_positions             19239 non-null  object 
 2   overall                      19239 non-null  int64  
 3   potential                    19239 non-null  int64  
 4   value_eur                    19165 non-null  float64
 5   wage_eur                     19178 non-null  float64
 6   age                          19239 non-null  int64  
 7   height_cm                    19239 non-null  int64  
 8   weight_kg                    19239 non-null  int64  
 9   club_name                    19178 non-null  object 
 10  league_name                  19178 non-null  object 
 11  league_level                 19178 non-null  float64
 12  club_position                19178 non-null  object 
 13  club_contract_va

In [177]:
categorical_data1 = pd.DataFrame()
numerical_data1 = pd.DataFrame()


In [178]:
for columns in new_22.columns:
    if new_22[columns].dtype == 'object':
        categorical_data1[columns] = new_22[columns]
    else:
        numerical_data1[columns] = new_22[columns]

In [179]:
categorical_data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 36 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   short_name        19239 non-null  object
 1   player_positions  19239 non-null  object
 2   club_name         19178 non-null  object
 3   league_name       19178 non-null  object
 4   club_position     19178 non-null  object
 5   nationality_name  19239 non-null  object
 6   preferred_foot    19239 non-null  object
 7   work_rate         19239 non-null  object
 8   player_traits     9398 non-null   object
 9   ls                19239 non-null  object
 10  st                19239 non-null  object
 11  rs                19239 non-null  object
 12  lw                19239 non-null  object
 13  lf                19239 non-null  object
 14  cf                19239 non-null  object
 15  rf                19239 non-null  object
 16  rw                19239 non-null  object
 17  lam         

In [180]:
numerical_data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 53 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      19239 non-null  int64  
 1   potential                    19239 non-null  int64  
 2   value_eur                    19165 non-null  float64
 3   wage_eur                     19178 non-null  float64
 4   age                          19239 non-null  int64  
 5   height_cm                    19239 non-null  int64  
 6   weight_kg                    19239 non-null  int64  
 7   league_level                 19178 non-null  float64
 8   club_contract_valid_until    19178 non-null  float64
 9   weak_foot                    19239 non-null  int64  
 10  skill_moves                  19239 non-null  int64  
 11  international_reputation     19239 non-null  int64  
 12  release_clause_eur           18063 non-null  float64
 13  pace            

In [181]:
print(numerical_data1.isnull().sum())

overall                           0
potential                         0
value_eur                        74
wage_eur                         61
age                               0
height_cm                         0
weight_kg                         0
league_level                     61
club_contract_valid_until        61
weak_foot                         0
skill_moves                       0
international_reputation          0
release_clause_eur             1176
pace                           2132
shooting                       2132
passing                        2132
dribbling                      2132
defending                      2132
physic                         2132
attacking_crossing                0
attacking_finishing               0
attacking_heading_accuracy        0
attacking_short_passing           0
attacking_volleys                 0
skill_dribbling                   0
skill_curve                       0
skill_fk_accuracy                 0
skill_long_passing          

In [183]:
numerical_data1.ffill(axis=0, inplace=True)

In [184]:
print(numerical_data1.isnull().sum())

overall                        0
potential                      0
value_eur                      0
wage_eur                       0
age                            0
height_cm                      0
weight_kg                      0
league_level                   0
club_contract_valid_until      0
weak_foot                      0
skill_moves                    0
international_reputation       0
release_clause_eur             0
pace                           0
shooting                       0
passing                        0
dribbling                      0
defending                      0
physic                         0
attacking_crossing             0
attacking_finishing            0
attacking_heading_accuracy     0
attacking_short_passing        0
attacking_volleys              0
skill_dribbling                0
skill_curve                    0
skill_fk_accuracy              0
skill_long_passing             0
skill_ball_control             0
movement_acceleration          0
movement_s

In [185]:
categorical_data1.ffill(axis=0, inplace=True)

In [186]:
print(categorical_data1.isnull().sum())

short_name          0
player_positions    0
club_name           0
league_name         0
club_position       0
nationality_name    0
preferred_foot      0
work_rate           0
player_traits       0
ls                  0
st                  0
rs                  0
lw                  0
lf                  0
cf                  0
rf                  0
rw                  0
lam                 0
cam                 0
ram                 0
lm                  0
lcm                 0
cm                  0
rcm                 0
rm                  0
lwb                 0
ldm                 0
cdm                 0
rdm                 0
rwb                 0
lb                  0
lcb                 0
cb                  0
rcb                 0
rb                  0
gk                  0
dtype: int64


In [187]:
encoded_categroical_data1 = categorical_data1.copy()

In [188]:
for columns in categorical_data1.columns:
    encoded_categroical_data1[columns] = encoder.fit_transform(categorical_data1[columns])

In [189]:
encoded_categroical_data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 36 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   short_name        19239 non-null  int64
 1   player_positions  19239 non-null  int64
 2   club_name         19239 non-null  int64
 3   league_name       19239 non-null  int64
 4   club_position     19239 non-null  int64
 5   nationality_name  19239 non-null  int64
 6   preferred_foot    19239 non-null  int64
 7   work_rate         19239 non-null  int64
 8   player_traits     19239 non-null  int64
 9   ls                19239 non-null  int64
 10  st                19239 non-null  int64
 11  rs                19239 non-null  int64
 12  lw                19239 non-null  int64
 13  lf                19239 non-null  int64
 14  cf                19239 non-null  int64
 15  rf                19239 non-null  int64
 16  rw                19239 non-null  int64
 17  lam               19239 non-nul

In [190]:
encoded_categroical_data1

,short_name,player_positions,club_name,league_name,club_position,nationality_name,preferred_foot,work_rate,player_traits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,10117,604,470,19,25,6,0,7,179,225,225,225,137,131,131,131,137,251,251,251,234,210,210,210,234,117,119,119,119,117,99,56,56,56,99,18
1,14468,635,241,21,27,123,1,2,930,227,227,227,133,128,128,128,133,245,245,245,224,186,186,186,224,107,131,131,131,107,99,105,105,105,99,18
2,3288,658,412,17,27,124,1,1,806,226,226,226,135,129,129,129,135,245,245,245,230,177,177,177,230,102,88,88,88,102,93,68,68,68,93,20
3,13097,372,470,19,14,20,1,2,399,213,213,213,136,128,128,128,136,249,249,249,232,195,195,195,232,123,113,113,113,123,104,56,56,56,104,20
4,9005,168,411,17,19,13,1,0,410,213,213,213,135,127,127,127,135,249,249,249,232,211,211,211,232,181,217,217,217,181,168,164,164,164,168,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,16139,98,688,6,28,30,1,8,291,40,40,40,32,30,30,30,32,47,47,47,46,48,48,48,46,45,47,47,47,45,49,46,46,46,49,9
19235,2947,167,212,39,21,126,1,8,291,41,41,41,33,31,31,31,33,48,48,48,49,49,49,49,49,48,47,47,47,48,49,42,42,42,49,7
19236,12809,167,291,39,21,126,1,8,291,41,41,41,32,31,31,31,32,48,48,48,47,49,49,49,47,44,47,47,47,44,47,44,44,44,47,3
19237,10276,635,291,39,21,126,1,8,291,44,44,44,33,32,32,32,33,46,46,46,43,35,35,35,43,25,15,15,15,25,25,11,11,11,25,9


In [192]:
clean_players_22_data = pd.concat([numerical_data1, encoded_categroical_data1], axis=1)

In [193]:
clean_players_22_data

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_level,club_contract_valid_until,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,short_name,player_positions,club_name,league_name,club_position,nationality_name,preferred_foot,work_rate,player_traits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,93,93,78000000.0,320000.0,34,170,72,1.0,2023.0,4,4,5,144300000.0,85.0,92.0,91.0,95.0,34.0,65.0,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96,20,35,24,6,11,15,14,8,10117,604,470,19,25,6,0,7,179,225,225,225,137,131,131,131,137,251,251,251,234,210,210,210,234,117,119,119,119,117,99,56,56,56,99,18
1,92,92,119500000.0,270000.0,32,185,81,1.0,2023.0,4,4,5,197200000.0,78.0,92.0,79.0,86.0,44.0,82.0,71,95,90,85,89,85,79,85,70,88,77,79,77,93,82,90,85,76,86,87,81,49,95,81,90,88,35,42,19,15,6,12,8,10,14468,635,241,21,27,123,1,2,930,227,227,227,133,128,128,128,133,245,245,245,224,186,186,186,224,107,131,131,131,107,99,105,105,105,99,18
2,91,91,45000000.0,270000.0,36,187,83,1.0,2023.0,4,5,5,83300000.0,87.0,94.0,80.0,88.0,34.0,75.0,87,95,90,80,86,88,81,84,77,88,85,88,86,94,74,94,95,77,77,93,63,29,95,76,88,95,24,32,24,7,11,15,14,11,3288,658,412,17,27,124,1,1,806,226,226,226,135,129,129,129,135,245,245,245,230,177,177,177,230,102,88,88,88,102,93,68,68,68,93,20
3,91,91,129000000.0,270000.0,29,175,68,1.0,2025.0,5,5,5,238700000.0,91.0,83.0,86.0,94.0,37.0,63.0,85,83,63,86,86,95,88,87,81,95,93,89,96,89,84,80,64,81,53,81,63,37,86,90,93,93,35,32,29,9,9,15,15,11,13097,372,470,19,14,20,1,2,399,213,213,213,136,128,128,128,136,249,249,249,232,195,195,195,232,123,113,113,113,123,104,56,56,56,104,20
4,91,91,125500000.0,350000.0,30,181,70,1.0,2025.0,5,4,4,232200000.0,76.0,86.0,93.0,88.0,64.0,78.0,94,82,55,94,82,88,85,83,93,91,76,76,79,91,78,91,63,89,74,91,76,66,88,94,83,89,68,65,53,15,13,5,10,13,9005,168,411,17,19,13,1,0,410,213,213,213,135,127,127,127,135,249,249,249,232,211,211,211,232,181,217,217,217,181,168,164,164,164,168,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47,52,70000.0,1000.0,22,180,64,1.0,2021.0,3,2,1,114000.0,58.0,35.0,46.0,48.0,42.0,49.0,46,32,48,50,30,45,33,38,48,49,56,60,55,53,70,46,62,51,46,30,52,42,38,43,42,37,38,43,48,6,10,5,15,13,16139,98,688,6,28,30,1,8,291,40,40,40,32,30,30,30,32,47,47,47,46,48,48,48,46,45,47,47,47,45,49,46,46,46,49,9
19235,47,59,110000.0,500.0,19,175,70,1.0,2021.0,3,2,1,193000.0,59.0,39.0,50.0,46.0,41.0,51.0,54,33,46,51,32,41,53,31,50,42,60,58,64,49,69,49,47,63,47,38,49,39,51,49,44,47,37,44,47,11,12,6,8,10,2947,167,212,39,21,126,1,8,291,41,41,41,33,31,31,31,33,48,48,48,49,49,49,49,49,48,47,47,47,48,49,42,42,42,49,7
19236,47,55,100000.0,500.0,21,178,72,1.0,2021.0,3,2,1,175000.0,60.0,37.0,45.0,49.0,41.0,52.0,39,32,43,49,37,47,37,37,49,49,60,60,58,46,59,50,57,56,50,34,51,38,45,46,39,36,38,44,48,8,6,7,10,6,12809,167,291,39,21,126,1,8,291,41,41,41,32,31,31,31,32,48,48,48,47,49,49,49,47,

In [194]:
selected_features

['movement_reactions',
 'potential',
 'passing',
 'wage_eur',
 'dribbling',
 'value_eur',
 'player_positions']

In [195]:
features = clean_players_22_data[selected_features]

In [196]:
features

,movement_reactions,potential,passing,wage_eur,dribbling,value_eur,player_positions
0,94,93,91.0,320000.0,95.0,78000000.0,604
1,93,92,79.0,270000.0,86.0,119500000.0,635
2,94,91,80.0,270000.0,88.0,45000000.0,658
3,89,91,86.0,270000.0,94.0,129000000.0,372
4,91,91,93.0,350000.0,88.0,125500000.0,168
...,...,...,...,...,...,...,...
19234,53,52,46.0,1000.0,48.0,70000.0,98
19235,49,59,50.0,500.0,46.0,110000.0,167
19236,46,55,45.0,500.0,49.0,100000.0,167
19237,48,60,36.0,500.0,48.0,110000.0,635


# Testing with new data

In [205]:
original_data = clean_players_22_data['overall']

In [197]:
features_scaled = scaler.transform(features)

In [202]:
random_forest_prediction = pd.DataFrame(imporoved_rfregressor.predict(features_scaled))
random_forest_prediction

,0
0,92.491667
1,91.441667
2,90.516667
3,90.841667
4,90.916667
...,...
19234,49.250000
19235,51.975000
19236,51.391667
19237,49.558333


In [203]:
xgboost_prediction = pd.DataFrame(xgb_final_model.predict(features_scaled))
xgboost_prediction

,0
0,91.528297
1,89.247635
2,89.647278
3,90.599373
4,90.624008
...,...
19234,50.128685
19235,51.360977
19236,49.464977
19237,49.678249


In [204]:
gradient_boost_prediction = pd.DataFrame(gradient_boost_final_model.predict(features_scaled))
gradient_boost_prediction

,0
0,91.212543
1,89.263698
2,89.416077
3,90.380471
4,90.250801
...,...
19234,49.735963
19235,51.754731
19236,50.351226
19237,50.779289


In [206]:
rmse_random_forest = np.sqrt(mean_squared_error(original_data, random_forest_prediction))
print("Random Forest RMSE:", rmse_random_forest)

Random Forest RMSE: 1.374750646460952


In [207]:
rmse_xgboost = np.sqrt(mean_squared_error(original_data, xgboost_prediction))
print("XGBoost RMSE:", rmse_xgboost)

XGBoost RMSE: 2.1145614673200153


In [208]:
rmse_gradient_boost = np.sqrt(mean_squared_error(original_data, gradient_boost_prediction))
print("Gradient Boost RMSE:", rmse_gradient_boost)

Gradient Boost RMSE: 2.2140223205771377


# Saving chosen model:

Chosen Model: ***Random Forest Regressor***

In [210]:
saved_directory = '/content/drive/My Drive/FIFA23_PREDICTIONMODEL/rfregressor.pk1'
with open(saved_directory, 'wb') as file:
    pickle.dump(imporoved_rfregressor, file)